In [44]:
import pandas as pd
file = 'Data/NC Monthly Precipitation Data.xlsx'
NCdata = pd.ExcelFile(file)
NCdata.sheet_names

['Raleigh',
 'Greensboro',
 'Fayetteville',
 'Albemarle',
 'Arcola',
 'Asheboro',
 'Burlington',
 'Carthage',
 'Chapel Hill',
 'Clayton',
 'Dunn',
 'Durham',
 'Enfield',
 'Fort Bragg',
 'Graham',
 'Greensboro AP',
 'Henderson 2 NNW',
 'Jackson Springs 5',
 'Laurinburg',
 'Louisburg',
 'Raleigh AP',
 'Randleman',
 'Roanoke Rapids',
 'Rougemont',
 'Siler City',
 'Smithfield',
 'Tarboro',
 'Wilson',
 'Yadkin College']

In [45]:
#Have an issue with the last 5 rows of data being included into the dataframe and I don't want them. Need to 
#figure out a way to import without them or to slice them off. 

ncdata = NCdata.parse('Raleigh', skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
ncdata = ncdata.dropna()
ncdata = ncdata.set_index('Year')
ncdata = ncdata.drop(['Mean','Max', 'Min'])
ncdata = ncdata.reset_index()
ncdata1 = pd.melt(ncdata, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
'Sep','Oct','Nov','Dec'], var_name='Month', value_name='Raleigh')
ncdata1.iloc[:,2] = pd.to_numeric(ncdata1.iloc[:,2], errors = 'coerce')
print(ncdata1.tail())
ncdata1.info()

      Year Month  Raleigh
1591  2015   Dec     6.07
1592  2016   Dec     2.19
1593  2017   Dec     2.29
1594  2018   Dec     6.23
1595  2019   Dec      NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 3 columns):
Year       1596 non-null int64
Month      1596 non-null object
Raleigh    1587 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 37.5+ KB


In [46]:
for i in range(1,6):
    to_merge = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    to_merge = to_merge.dropna()
    to_merge = to_merge.set_index('Year')
    to_merge = to_merge.drop(['Mean','Max', 'Min'])
    to_merge = to_merge.reset_index()
    to_merge1 = pd.melt(to_merge, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
    'Sep','Oct','Nov','Dec'], var_name='Month') 
    to_merge1.iloc[:,2] = pd.to_numeric(to_merge1.iloc[:,2], errors = 'coerce')
    ncdata1 = pd.merge(ncdata1, to_merge1, on = ['Year','Month'], how = 'outer')
print(ncdata1.tail())

      Year Month  Raleigh  value_x  value_y  value_x  value_y  value
1591  2015   Dec     6.07     6.65     6.51    10.07     5.40  11.11
1592  2016   Dec     2.19     1.70     2.64     3.46     1.68   2.03
1593  2017   Dec     2.29     1.60     2.97     1.70      NaN   2.00
1594  2018   Dec     6.23     6.99     6.00     8.43      NaN   6.41
1595  2019   Dec      NaN      NaN      NaN      NaN      NaN    NaN


In [47]:
for i in range(6,13):
    to_merge2 = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    to_merge2 = to_merge2.dropna()
    to_merge2 = to_merge2.set_index('Year')
    to_merge2 = to_merge2.drop(['Mean','Max', 'Min'])
    to_merge2 = to_merge2.reset_index()
    to_merge3 = pd.melt(to_merge2, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
'Sep','Oct','Nov','Dec'], var_name='Month') #value_name= NCdata.sheet_names[i]) 
    to_merge3.iloc[:,2] = pd.to_numeric(to_merge3.iloc[:,2], errors = 'coerce')
    ncdata1 = pd.merge(ncdata1, to_merge3, on = ['Year','Month'], how = 'outer')
ncdata1.tail()

,Year,Month,Raleigh,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y
1591,2015,Dec,6.07,6.65,6.51,10.07,5.40,11.11,NaN,5.54,8.66,7.25,6.22,NaN,NaN
1592,2016,Dec,2.19,1.70,2.64,3.46,1.68,2.03,NaN,NaN,NaN,2.78,2.49,NaN,NaN
1593,2017,Dec,2.29,1.60,2.97,1.70,NaN,2.00,NaN,2.93,2.12,2.90,2.96,NaN,NaN
1594,2018,Dec,6.23,6.99,6.00,8.43,NaN,6.41,NaN,NaN,NaN,9.24,7.22,NaN,NaN
1595,2019,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
for i in range(13,20):
    to_merge4 = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    to_merge4 = to_merge4.dropna()
    to_merge4 = to_merge4.set_index('Year')
    to_merge4 = to_merge4.drop(['Mean','Max', 'Min'])
    to_merge4 = to_merge4.reset_index()
    to_merge5 = pd.melt(to_merge4, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
'Sep','Oct','Nov','Dec'], var_name='Month') #value_name= NCdata.sheet_names[i]) 
    to_merge5.iloc[:,2] = pd.to_numeric(to_merge5.iloc[:,2], errors = 'coerce')
    ncdata1 = pd.merge(ncdata1, to_merge5, on = ['Year','Month'], how = 'outer')
ncdata1.tail()

,Year,Month,Raleigh,value_x,value_y,value_x,value_y,value_x,value_y,value_x,...,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value
1591,2015,Dec,6.07,6.65,6.51,10.07,5.40,11.11,NaN,5.54,...,6.22,NaN,NaN,NaN,7.85,6.65,NaN,8.97,6.96,NaN
1592,2016,Dec,2.19,1.70,2.64,3.46,1.68,2.03,NaN,NaN,...,2.49,NaN,NaN,NaN,1.90,1.70,NaN,3.38,2.65,NaN
1593,2017,Dec,2.29,1.60,2.97,1.70,NaN,2.00,NaN,2.93,...,2.96,NaN,NaN,NaN,1.62,1.60,1.88,3.36,3.20,2.24
1594,2018,Dec,6.23,6.99,6.00,8.43,NaN,6.41,NaN,NaN,...,7.22,NaN,NaN,NaN,7.39,6.99,4.44,7.55,7.06,4.16
1595,2019,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
for i in range(20,26):
    to_merge6 = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    to_merge6 = to_merge6.dropna()
    to_merge6 = to_merge6.set_index('Year')
    to_merge6 = to_merge6.drop(['Mean','Max', 'Min'])
    to_merge6 = to_merge6.reset_index()
    to_merge7 = pd.melt(to_merge6, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
'Sep','Oct','Nov','Dec'], var_name='Month') #value_name= NCdata.sheet_names[i]) 
    to_merge7.iloc[:,2] = pd.to_numeric(to_merge7.iloc[:,2], errors = 'coerce')
    ncdata1 = pd.merge(ncdata1, to_merge7, on = ['Year','Month'], how = 'outer')
ncdata1.tail()

,Year,Month,Raleigh,value_x,value_y,value_x,value_y,value_x,value_y,value_x,...,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value
1591,2015,Dec,6.07,6.65,6.51,10.07,5.40,11.11,NaN,5.54,...,NaN,8.97,6.96,NaN,6.07,8.93,4.48,NaN,8.12,6.85
1592,2016,Dec,2.19,1.70,2.64,3.46,1.68,2.03,NaN,NaN,...,NaN,3.38,2.65,NaN,2.19,1.65,NaN,NaN,2.04,2.99
1593,2017,Dec,2.29,1.60,2.97,1.70,NaN,2.00,NaN,2.93,...,1.88,3.36,3.20,2.24,2.29,1.94,2.09,NaN,NaN,2.82
1594,2018,Dec,6.23,6.99,6.00,8.43,NaN,6.41,NaN,NaN,...,4.44,7.55,7.06,4.16,6.23,4.98,4.62,NaN,NaN,5.82
1595,2019,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
colnames = ['Year', 'Month']
n = NCdata.sheet_names
colnames = colnames + n
print(colnames)

['Year', 'Month', 'Raleigh', 'Greensboro', 'Fayetteville', 'Albemarle', 'Arcola', 'Asheboro', 'Burlington', 'Carthage', 'Chapel Hill', 'Clayton', 'Dunn', 'Durham', 'Enfield', 'Fort Bragg', 'Graham', 'Greensboro AP', 'Henderson 2 NNW', 'Jackson Springs 5', 'Laurinburg', 'Louisburg', 'Raleigh AP', 'Randleman', 'Roanoke Rapids', 'Rougemont', 'Siler City', 'Smithfield', 'Tarboro', 'Wilson', 'Yadkin College']


In [51]:
for i in range(26,29):
    to_merge8 = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    to_merge8 = to_merge8.dropna()
    to_merge8 = to_merge8.set_index('Year')
    to_merge8 = to_merge8.drop(['Mean','Max', 'Min'])
    to_merge8 = to_merge8.reset_index()
    to_merge9 = pd.melt(to_merge8, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
'Sep','Oct','Nov','Dec'], var_name='Month') #value_name= NCdata.sheet_names[i]) 
    to_merge9.iloc[:,2] = pd.to_numeric(to_merge9.iloc[:,2], errors = 'coerce')
    ncdata1 = pd.merge(ncdata1, to_merge9, on = ['Year','Month'], how = 'outer')
ncdata1.columns = colnames
ncdata1.tail()

,Year,Month,Raleigh,Greensboro,Fayetteville,Albemarle,Arcola,Asheboro,Burlington,Carthage,...,Louisburg,Raleigh AP,Randleman,Roanoke Rapids,Rougemont,Siler City,Smithfield,Tarboro,Wilson,Yadkin College
1795,1882,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1796,1883,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1797,1884,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798,1885,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1799,1886,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
ncdata1["Month"] = pd.to_datetime(ncdata1.Month, format='%b', errors='coerce').dt.month
ncdata1["Year"] = pd.to_datetime(ncdata1.Year, format='%Y', errors='coerce').dt.year
print(ncdata1.head())
ncdata1.info()

   Year  Month  Raleigh  Greensboro  Fayetteville  Albemarle  Arcola  \
0  1887      1     3.04         NaN           NaN        NaN     NaN   
1  1888      1     3.98         NaN           NaN        NaN     NaN   
2  1889      1     6.02         NaN           NaN        NaN     NaN   
3  1890      1     0.83         NaN           NaN        NaN     NaN   
4  1891      1     3.96         NaN           NaN        NaN     NaN   

   Asheboro  Burlington  Carthage  ...  Louisburg  Raleigh AP  Randleman  \
0       NaN         NaN       NaN  ...        NaN         NaN        NaN   
1       NaN         NaN       NaN  ...        NaN         NaN        NaN   
2       NaN         NaN       NaN  ...        NaN         NaN        NaN   
3       NaN         NaN       NaN  ...        NaN         NaN        NaN   
4       NaN         NaN       NaN  ...        NaN         NaN        NaN   

   Roanoke Rapids  Rougemont  Siler City  Smithfield  Tarboro  Wilson  \
0             NaN        NaN         

In [53]:
ncdata1 = ncdata1.sort_values(['Year','Month'])
ncdata1.tail()

,Year,Month,Raleigh,Greensboro,Fayetteville,Albemarle,Arcola,Asheboro,Burlington,Carthage,...,Louisburg,Raleigh AP,Randleman,Roanoke Rapids,Rougemont,Siler City,Smithfield,Tarboro,Wilson,Yadkin College
1063,2019,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2019,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,2019,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1595,2019,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
ncdata1.to_excel('ncrainfalldata.xlsx')